In [1]:
%matplotlib inline

In [84]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from google_drive_downloader import GoogleDriveDownloader as gdd

import cv2

import os

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, UpSampling2D

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

from tensorflow.keras.callbacks import TensorBoard

# Barcode Image Segmentation

## Abstract

Barcodes are widely used to localize items on images. Numerous applications are available to find and recognize the barcodes. However they cannot manage cases such as targets with small scale, oclusions, shape deformations, noise and blurring. The most widely solutions require the barcode to be oriented in single directions and may fail in conditions that seem without problems.

This work attempts to solve the problem of barcode localization using a deep learning based segmentation approach.

## Datasets

In [86]:
# Change to the folder where the images datasets are loaded/present and where the 
WORKPLACE_FOLDER = "/tmp"

# Folder to store/read checkpoints for model training
CHECKPOINTS_DIR = "checkpoints"

# Image size for the input
IMAGE_SIZE = (224, 224)

# Image size + channels for the input
INPUT_IMAGE_SIZE = (IMAGE_SIZE[0], IMAGE_SIZE[1], 3)

RESUME_MODELS = True
TRAIN_MODELS = False

BATCH_SIZE = 32
NUM_EPOCHS = 60

In total five datasets are used - two to train the neural network and three to test.

First three datasets are downloaded from http://artelab.dista.uninsubria.it/downloads/datasets/barcode/medium_barcode_1d/medium_barcode_1d.html (ARTELAB) [[1](#ref_1)]

The masks for the segments containing barcodes were created using the [pyzbar](https://pypi.org/project/pyzbar/pyzbar) library. For this purpose the image was rotated with predefined steps and the obtained from [pyzbar](https://pypi.org/project/pyzbar/pyzbar) points were used to set the required segments.

It has to be noted that the [pyzbar](https://pypi.org/project/pyzbar/pyzbar) did not produced reliable results even for some images that seemed without defects. For this reason not all the images from [ARTELAB](http://artelab.dista.uninsubria.it/downloads/datasets/barcode/medium_barcode_1d/medium_barcode_1d.html) were used. Only the successfully generated masks were applied.

### Dataset 1

Pictures of barcodes taken from devices with autofocus - first subset.
Contains 122 images (originals + masks) with zipped size 38,5 MB 

In [4]:
gdd.download_file_from_google_drive(file_id="18MFEr2iekIojLwEhzswOIyW_Fp8CNQA5",
                                    dest_path=os.path.join(WORKPLACE_FOLDER, "Dataset1.zip"),
                                    unzip=True)

Unzipping...Done.


### Dataset 2

Pictures of barcodes taken from devices with autofocus - first subset.
Contains 76 images (originals + masks) with zipped size 124,5 MB

In [5]:
gdd.download_file_from_google_drive(file_id="1SHJi744MZV40Mp38m6RW8PeuQktbEt6u",
                                    dest_path=os.path.join(WORKPLACE_FOLDER, "Dataset2.zip"),
                                    unzip=True)

Unzipping...Done.


### Dataset 3

Pictures of barcodes taken from devices without autofocus. Contains 61 images (originals + masks) with zipped size 13,6 MB 

In [6]:
gdd.download_file_from_google_drive(file_id="1ybx4TiciMoQcpVi3fAzZoUOuSg2WPrvI",
                                    dest_path=os.path.join(WORKPLACE_FOLDER, "Dataset3.zip"),
                                    unzip=True)

Unzipping...Done.


### Dataset 4

Downloaded from https://github.com/rohrlaf/SlaRle.js/tree/master/Muenster%20BarcodeDB and referenced as Muenster BarodeDB [[2](#ref_2)]. Masks were prepared following the above mentioned procedure with [pyzbar](https://pypi.org/project/pyzbar/pyzbar) 

Contains 863 images (originals + masks) with zipped size 46,9 MB 

In [7]:
gdd.download_file_from_google_drive(file_id="1gfxKTaG7tHDDK5fPQW6PH-Zcbx7KPXzO",
                                    dest_path=os.path.join(WORKPLACE_FOLDER, "Dataset4.zip"),
                                    unzip=True)

Unzipping...Done.


### Dataset 5

Downloaded from http://artelab.dista.uninsubria.it/downloads/datasets/barcode/hough_barcode_1d/hough_barcode_1d.html (ARTELAB) [[3](#ref_3)] Referenced as dataset no.2 plain (1d_barcode_extended_plain.zip) contains only the images and the detection masks. Masks had to be adjusted to be grayscale one channel images.

Contains 365 images (originals + masks) with zipped size 37,5 MB

In [8]:
gdd.download_file_from_google_drive(file_id="1rNi26q-iq5Q4BtrIOT-pDSleCtKzw3pk",
                                    dest_path=os.path.join(WORKPLACE_FOLDER, "Dataset5.zip"),
                                    unzip=True)

Unzipping...Done.


In [9]:
def get_all_filenames(base_dir):
    """
    Returns the filenames for barcodes and masks
    Assumes the following structure:
    |---base_dir
    | |---Original
    | | |---image1.jpg
    | | |---image2.jpg
    | |---Detection
    | | |---image1.png
    | | |---image2.png
    
    :param base_dir: directories where image databse is stored
    
    """
    
    filenames = {}
    filenames["Original"] = []
    filenames["Detection"] = []
    
    # True if image is to be rotated counter-clockwise by 90 degrees.
    filenames["Rotation"] = []
    
    original_directory = os.path.join(base_dir,"Original")
    detection_directory = os.path.join(base_dir,"Detection")
    for file_name in os.listdir(original_directory):
        if file_name.endswith(".jpg"):
            original_name = os.path.join(original_directory, file_name)
            base_filename=os.path.splitext(file_name)[0]
            detection_name = os.path.join(detection_directory, base_filename + ".png")
            if os.path.exists(detection_name):
                filenames["Rotation"].append(True)
                filenames["Original"].append(original_name)
                filenames["Detection"].append(detection_name)
                filenames["Rotation"].append(False)
                filenames["Original"].append(original_name)
                filenames["Detection"].append(detection_name)
        
    return pd.DataFrame(filenames)

In [10]:
filenames_dataset_1 = get_all_filenames(os.path.join(WORKPLACE_FOLDER, "Dataset1"))
filenames_dataset_2 = get_all_filenames(os.path.join(WORKPLACE_FOLDER, "Dataset2"))
filenames_dataset_3 = get_all_filenames(os.path.join(WORKPLACE_FOLDER, "Dataset3"))
filenames_dataset_4 = get_all_filenames(os.path.join(WORKPLACE_FOLDER, "Dataset4"))
filenames_dataset_5 = get_all_filenames(os.path.join(WORKPLACE_FOLDER, "Dataset5"))

In [11]:
assert(filenames_dataset_1.shape[0] == 244)

In [12]:
assert(filenames_dataset_2.shape[0] == 152)

In [13]:
assert(filenames_dataset_3.shape[0] == 122)

In [14]:
assert(filenames_dataset_4.shape[0] == 1726)

In [15]:
assert(filenames_dataset_5.shape[0] == 730)

## Preparation of images

In [16]:
def preprocess_image(x):
    # Normalize
    x = tf.cast(x, tf.float32) / 255.0

    # 'RGB'->'BGR'
    x = x[..., ::-1]
    return x

def read_and_prepare_image(original_image_filename, mask_image_filename, is_rotated):
    # Get images
    original_image = tf.io.read_file(original_image_filename)
    original_image_decoded = tf.image.decode_jpeg(original_image)
    mask_image = tf.io.read_file(mask_image_filename)
    mask_image_decoded = tf.image.decode_jpeg(mask_image)
    
    # Resize
    original_image_resized = tf.image.resize(original_image_decoded, IMAGE_SIZE)
    mask_image_resized = tf.image.resize(mask_image_decoded, IMAGE_SIZE)
    
    # Rotate
    if is_rotated:
        original_image_resized = tf.image.rot90(original_image_resized)
        mask_image_resized = tf.image.rot90(mask_image_resized)

    original_image_tensor = preprocess_image(original_image_resized)
    mask_image_tensor = preprocess_image(mask_image_resized)
    
    return original_image_tensor, mask_image_tensor

In [17]:
def initialize_tf_dataset(data, should_batch = True, should_repeat = True):
    dataset_initial = tf.data.Dataset.from_tensor_slices((data.Original.values, data.Detection.values, data.Rotation.values))
    dataset_mapped = dataset_initial.map(read_and_prepare_image)
    dataset_shuffled = dataset_mapped.shuffle(buffer_size = len(data))
    
    if should_batch:
        dataset = dataset_shuffled.batch(BATCH_SIZE)
    else:
        dataset = dataset_shuffled.batch(len(data))
        
    if should_repeat:
        dataset = dataset.repeat()
    return dataset

In [18]:
train_data_1 = initialize_tf_dataset(filenames_dataset_4)
train_data_2 = initialize_tf_dataset(filenames_dataset_5)

In [26]:
train_data_map = {
    train_data_1 : "train1",
    train_data_2 : "train2"
}

In [22]:
test_data_1 = initialize_tf_dataset(filenames_dataset_1, should_batch = False, should_repeat = False)
test_data_2 = initialize_tf_dataset(filenames_dataset_2, should_batch = False, should_repeat = False)
test_data_3 = initialize_tf_dataset(filenames_dataset_3, should_batch = False, should_repeat = False)

In [27]:
test_data_map = {
    test_data_1 : "test1",
    test_data_2 : "test2",
    test_data_3 : "test3"
}

In [24]:
for data in train_data_map.keys():
    for batch in data:
        assert(batch[0].shape[0] == BATCH_SIZE)
        assert(batch[0].shape[1] == INPUT_IMAGE_SIZE[0])
        assert(batch[0].shape[2] == INPUT_IMAGE_SIZE[1])
        assert(batch[0].shape[3] == INPUT_IMAGE_SIZE[2])
        assert(batch[1].shape[0] == BATCH_SIZE)
        assert(batch[1].shape[1] == INPUT_IMAGE_SIZE[0])
        assert(batch[1].shape[2] == INPUT_IMAGE_SIZE[1])
        assert(batch[1].shape[3] == 1)
        break

In [25]:
for data, filenames in zip(test_data_map.keys(),
                           [filenames_dataset_1, filenames_dataset_2, filenames_dataset_3]):
    for batch in data:
        assert(batch[0].shape[0] == filenames.shape[0])
        assert(batch[0].shape[1] == INPUT_IMAGE_SIZE[0])
        assert(batch[0].shape[2] == INPUT_IMAGE_SIZE[1])
        assert(batch[0].shape[3] == INPUT_IMAGE_SIZE[2])
        assert(batch[1].shape[0] == filenames.shape[0])
        assert(batch[1].shape[1] == INPUT_IMAGE_SIZE[0])
        assert(batch[1].shape[2] == INPUT_IMAGE_SIZE[1])
        assert(batch[1].shape[3] == 1)
        break

## Model

In [28]:
def get_model_2lw():
    model = Sequential([
        Input(INPUT_IMAGE_SIZE),
        Conv2D(8,(3,3), activation=tf.keras.activations.relu, padding='same'),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(16,(3,3), activation=tf.keras.activations.relu, padding='same'),
        BatchNormalization(),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(16, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(8, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(1, (1, 1), activation=tf.keras.activations.relu, padding='same')
    ])
    return model

In [29]:
def get_model_3lw():
    model = Sequential([
        Input(INPUT_IMAGE_SIZE),
        Conv2D(8,(3,3), activation=tf.keras.activations.relu, padding='same'),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(16,(3,3), activation=tf.keras.activations.relu, padding='same'),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(32,(3,3), activation=tf.keras.activations.relu, padding='same'),
        BatchNormalization(),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(32, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(16, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(8, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(1, (1, 1), activation=tf.keras.activations.relu, padding='same')
    ])
    return model

In [30]:
def get_model_4lw():
    model = Sequential([
        Input(INPUT_IMAGE_SIZE),
        Conv2D(8,(3,3), activation=tf.keras.activations.relu, padding='same'),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(16,(3,3), activation=tf.keras.activations.relu, padding='same'),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(32,(3,3), activation=tf.keras.activations.relu, padding='same'),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(64,(3,3), activation=tf.keras.activations.relu, padding='same'),
        BatchNormalization(),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(64, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(32, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(16, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(8, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(1, (1, 1), activation=tf.keras.activations.relu, padding='same')
    ])
    return model

In [31]:
def get_model_5lw():
    model = Sequential([
        Input(INPUT_IMAGE_SIZE),
        Conv2D(8,(3,3), activation=tf.keras.activations.relu, padding='same'),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(16,(3,3), activation=tf.keras.activations.relu, padding='same'),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(32,(3,3), activation=tf.keras.activations.relu, padding='same'),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(64,(3,3), activation=tf.keras.activations.relu, padding='same'),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(128,(3,3), activation=tf.keras.activations.relu, padding='same'),
        BatchNormalization(),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(128, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(64, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(32, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(16, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(8, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(1, (1, 1), activation=tf.keras.activations.relu, padding='same')
    ])
    return model

In [33]:
model_w_map = {
    get_model_2lw() : "model2lw",
    get_model_3lw() : "model3lw",
    get_model_4lw() : "model4lw",
    get_model_5lw() : "model5lw",
}

In [34]:
def get_model_2ln():
    model = Sequential([
        Input(INPUT_IMAGE_SIZE),
        Conv2D(4,(3,3), activation=tf.keras.activations.relu, padding='same'),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(8,(3,3), activation=tf.keras.activations.relu, padding='same'),
        BatchNormalization(),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(8, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(4, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(1, (1, 1), activation=tf.keras.activations.relu, padding='same')
    ])
    return model

In [35]:
def get_model_3ln():
    model = Sequential([
        Input(INPUT_IMAGE_SIZE),
        Conv2D(4,(3,3), activation=tf.keras.activations.relu, padding='same'),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(8,(3,3), activation=tf.keras.activations.relu, padding='same'),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(16,(3,3), activation=tf.keras.activations.relu, padding='same'),
        BatchNormalization(),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(16, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(8, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(4, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(1, (1, 1), activation=tf.keras.activations.relu, padding='same')
    ])
    return model

In [36]:
def get_model_4ln():
    model = Sequential([
        Input(INPUT_IMAGE_SIZE),
        Conv2D(4,(3,3), activation=tf.keras.activations.relu, padding='same'),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(8,(3,3), activation=tf.keras.activations.relu, padding='same'),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(16,(3,3), activation=tf.keras.activations.relu, padding='same'),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(32,(3,3), activation=tf.keras.activations.relu, padding='same'),
        BatchNormalization(),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(32, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(16, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(8, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(4, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(1, (1, 1), activation=tf.keras.activations.relu, padding='same')
    ])
    return model

In [37]:
def get_model_5ln():
    model = Sequential([
        Input(INPUT_IMAGE_SIZE),
        Conv2D(4,(3,3), activation=tf.keras.activations.relu, padding='same'),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(8,(3,3), activation=tf.keras.activations.relu, padding='same'),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(16,(3,3), activation=tf.keras.activations.relu, padding='same'),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(32,(3,3), activation=tf.keras.activations.relu, padding='same'),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(64,(3,3), activation=tf.keras.activations.relu, padding='same'),
        BatchNormalization(),
        MaxPooling2D((2,2), padding='same'),
        Conv2D(64, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(32, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(16, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(8, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(4, (3, 3), activation=tf.keras.activations.relu, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(1, (1, 1), activation=tf.keras.activations.relu, padding='same')
    ])
    return model

In [38]:
model_n_map = {
    get_model_2ln() : "model2ln",
    get_model_3ln() : "model3ln",
    get_model_4ln() : "model4ln",
    get_model_5ln() : "model5ln",
}

In [87]:
for model in model_w_map.keys():
    model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 224, 224, 8)       224       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 112, 112, 8)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 112, 112, 16)      1168      
_________________________________________________________________
batch_normalization_1 (Batch (None, 112, 112, 16)      64        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 56, 56, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 56, 56, 16)        2320      
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 112, 112, 16)     

In [88]:
for model in model_n_map.keys():
    model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 224, 224, 4)       112       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 112, 112, 4)       0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 112, 112, 8)       296       
_________________________________________________________________
batch_normalization_5 (Batch (None, 112, 112, 8)       32        
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 56, 56, 8)         0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 56, 56, 8)         584       
_________________________________________________________________
up_sampling2d_16 (UpSampling (None, 112, 112, 8)      

In [42]:
steps_per_epoch_train_1 = round(len(filenames_dataset_4) * 1.0 / BATCH_SIZE)
steps_per_epoch_train_2 = round(len(filenames_dataset_5) * 1.0 / BATCH_SIZE)

In [43]:
steps_per_epoch_train_1

54

In [44]:
steps_per_epoch_train_2

23

In [45]:
steps_per_epoch_map = {
    train_data_1 : steps_per_epoch_train_1,
    train_data_2 : steps_per_epoch_train_2
}

In [55]:
if not os.path.exists(CHECKPOINTS_DIR):
    os.makedirs(CHECKPOINTS_DIR)

In [85]:
for train_item in train_data_map.items():
    train_data = train_item[0]
    train_name = train_item[1]
    for model_item in {**model_w_map , **model_n_map}.items():
        model = model_item[0]
        model_name = model_item[1]
        checkpoint_basename = train_name + "_" + model_name
        checkpoint_path = os.path.join(CHECKPOINTS_DIR,checkpoint_basename)
        
        if not os.path.exists(checkpoint_path):
            os.makedirs(checkpoint_path)
        
        model_weights_file = os.path.join(checkpoint_path, "cp.ckpt")
        log_path = os.path.join(checkpoint_path, "log")
        
        # Callback to save the model's weights
        cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=model_weights_file,
                                                         save_weights_only=True)
        
        model.compile(optimizer=Adam(), loss=MeanSquaredError())
        
        needs_training = True
        if os.path.exists(model_weights_file) and RESUME_MODELS:
            model.load_weights(model_weights_file)
            needs_training = False
        
        if TRAIN_MODELS or needs_training:
            history = model.fit(train_data,
                                epochs = NUM_EPOCHS,
                                steps_per_epoch = steps_per_epoch_map[train_data],
                                callbacks = [TensorBoard(log_dir=log_path),
                                             cp_callback]
                               )

        
        


Train for 54 steps
Epoch 1/60
54/54 [==============================] - 36s 666ms/step - loss: 0.0431
Epoch 2/60
54/54 [==============================] - 36s 662ms/step - loss: 0.0194
Epoch 3/60
54/54 [==============================] - 36s 658ms/step - loss: 0.0155
Epoch 4/60
54/54 [==============================] - 35s 639ms/step - loss: 0.0132
Epoch 5/60
54/54 [==============================] - 34s 638ms/step - loss: 0.0120
Epoch 6/60
54/54 [==============================] - 34s 638ms/step - loss: 0.0109
Epoch 7/60
54/54 [==============================] - 34s 638ms/step - loss: 0.0103
Epoch 8/60
54/54 [==============================] - 34s 634ms/step - loss: 0.0096
Epoch 9/60
54/54 [==============================] - 36s 658ms/step - loss: 0.0093
Epoch 10/60
54/54 [==============================] - 35s 641ms/step - loss: 0.0089
Epoch 11/60
54/54 [==============================] - 35s 639ms/step - loss: 0.0085
Epoch 12/60
54/54 [==============================] - 34s 637ms/step - loss: 0

54/54 [==============================] - 38s 712ms/step - loss: 0.0049
Epoch 40/60
54/54 [==============================] - 38s 711ms/step - loss: 0.0049
Epoch 41/60
54/54 [==============================] - 38s 711ms/step - loss: 0.0048
Epoch 42/60
54/54 [==============================] - 38s 706ms/step - loss: 0.0047
Epoch 43/60
54/54 [==============================] - 38s 711ms/step - loss: 0.0049
Epoch 44/60
54/54 [==============================] - 41s 754ms/step - loss: 0.0049
Epoch 45/60
54/54 [==============================] - 43s 801ms/step - loss: 0.0047
Epoch 46/60
54/54 [==============================] - 43s 804ms/step - loss: 0.0046
Epoch 47/60
54/54 [==============================] - 45s 837ms/step - loss: 0.0046
Epoch 48/60
54/54 [==============================] - 45s 836ms/step - loss: 0.0047
Epoch 49/60
54/54 [==============================] - 45s 829ms/step - loss: 0.0047
Epoch 50/60
54/54 [==============================] - 46s 858ms/step - loss: 0.0046
Epoch 51/60
54/5

54/54 [==============================] - 43s 793ms/step - loss: 0.0932
Epoch 18/60
54/54 [==============================] - 43s 789ms/step - loss: 0.0932
Epoch 19/60
54/54 [==============================] - 43s 792ms/step - loss: 0.0932
Epoch 20/60
54/54 [==============================] - 43s 790ms/step - loss: 0.0932
Epoch 21/60
54/54 [==============================] - 43s 791ms/step - loss: 0.0931
Epoch 22/60
54/54 [==============================] - 43s 793ms/step - loss: 0.0932
Epoch 23/60
54/54 [==============================] - 43s 791ms/step - loss: 0.0932
Epoch 24/60
54/54 [==============================] - 43s 793ms/step - loss: 0.0932
Epoch 25/60
54/54 [==============================] - 43s 794ms/step - loss: 0.0931
Epoch 26/60
54/54 [==============================] - 43s 796ms/step - loss: 0.0932
Epoch 27/60
54/54 [==============================] - 43s 793ms/step - loss: 0.0932
Epoch 28/60
54/54 [==============================] - 43s 794ms/step - loss: 0.0931
Epoch 29/60
54/5

54/54 [==============================] - 23s 420ms/step - loss: 0.0088
Epoch 56/60
54/54 [==============================] - 23s 420ms/step - loss: 0.0087
Epoch 57/60
54/54 [==============================] - 23s 420ms/step - loss: 0.0089
Epoch 58/60
54/54 [==============================] - 23s 420ms/step - loss: 0.0088
Epoch 59/60
54/54 [==============================] - 23s 420ms/step - loss: 0.0087
Epoch 60/60
54/54 [==============================] - 23s 419ms/step - loss: 0.0087
Train for 54 steps
Epoch 1/60
54/54 [==============================] - 25s 471ms/step - loss: 0.0401
Epoch 2/60
54/54 [==============================] - 25s 458ms/step - loss: 0.0211
Epoch 3/60
54/54 [==============================] - 25s 459ms/step - loss: 0.0161
Epoch 4/60
54/54 [==============================] - 25s 457ms/step - loss: 0.0134
Epoch 5/60
54/54 [==============================] - 25s 457ms/step - loss: 0.0120
Epoch 6/60
54/54 [==============================] - 25s 460ms/step - loss: 0.0111
Epo

54/54 [==============================] - 26s 479ms/step - loss: 0.0051
Epoch 34/60
54/54 [==============================] - 26s 480ms/step - loss: 0.0050
Epoch 35/60
54/54 [==============================] - 26s 478ms/step - loss: 0.0050
Epoch 36/60
54/54 [==============================] - 26s 481ms/step - loss: 0.0050
Epoch 37/60
54/54 [==============================] - 26s 482ms/step - loss: 0.0050
Epoch 38/60
54/54 [==============================] - 26s 478ms/step - loss: 0.0049
Epoch 39/60
54/54 [==============================] - 26s 478ms/step - loss: 0.0049
Epoch 40/60
54/54 [==============================] - 26s 479ms/step - loss: 0.0048
Epoch 41/60
54/54 [==============================] - 26s 478ms/step - loss: 0.0048
Epoch 42/60
54/54 [==============================] - 26s 479ms/step - loss: 0.0048
Epoch 43/60
54/54 [==============================] - 26s 478ms/step - loss: 0.0047
Epoch 44/60
54/54 [==============================] - 26s 479ms/step - loss: 0.0047
Epoch 45/60
54/5

23/23 [==============================] - 14s 625ms/step - loss: 0.0143
Epoch 12/60
23/23 [==============================] - 14s 622ms/step - loss: 0.0144
Epoch 13/60
23/23 [==============================] - 14s 627ms/step - loss: 0.0163
Epoch 14/60
23/23 [==============================] - 14s 624ms/step - loss: 0.0145
Epoch 15/60
23/23 [==============================] - 14s 624ms/step - loss: 0.0132
Epoch 16/60
23/23 [==============================] - 14s 624ms/step - loss: 0.0126
Epoch 17/60
23/23 [==============================] - 14s 625ms/step - loss: 0.0124
Epoch 18/60
23/23 [==============================] - 14s 623ms/step - loss: 0.0126
Epoch 19/60
23/23 [==============================] - 14s 626ms/step - loss: 0.0125
Epoch 20/60
23/23 [==============================] - 14s 621ms/step - loss: 0.0125
Epoch 21/60
23/23 [==============================] - 14s 626ms/step - loss: 0.0125
Epoch 22/60
23/23 [==============================] - 14s 626ms/step - loss: 0.0126
Epoch 23/60
23/2

23/23 [==============================] - 16s 700ms/step - loss: 0.0047
Epoch 50/60
23/23 [==============================] - 16s 697ms/step - loss: 0.0046
Epoch 51/60
23/23 [==============================] - 16s 699ms/step - loss: 0.0046
Epoch 52/60
23/23 [==============================] - 16s 695ms/step - loss: 0.0045
Epoch 53/60
23/23 [==============================] - 16s 699ms/step - loss: 0.0046
Epoch 54/60
23/23 [==============================] - 16s 698ms/step - loss: 0.0045
Epoch 55/60
23/23 [==============================] - 16s 698ms/step - loss: 0.0046
Epoch 56/60
23/23 [==============================] - 16s 696ms/step - loss: 0.0043
Epoch 57/60
23/23 [==============================] - 16s 694ms/step - loss: 0.0043
Epoch 58/60
23/23 [==============================] - 16s 698ms/step - loss: 0.0043
Epoch 59/60
23/23 [==============================] - 16s 697ms/step - loss: 0.0042
Epoch 60/60
23/23 [==============================] - 16s 702ms/step - loss: 0.0052
Train for 23 ste

23/23 [==============================] - 18s 778ms/step - loss: 0.1252
Epoch 28/60
23/23 [==============================] - 18s 779ms/step - loss: 0.1252
Epoch 29/60
23/23 [==============================] - 18s 779ms/step - loss: 0.1252
Epoch 30/60
23/23 [==============================] - 18s 779ms/step - loss: 0.1251
Epoch 31/60
23/23 [==============================] - 18s 781ms/step - loss: 0.1250
Epoch 32/60
23/23 [==============================] - 18s 780ms/step - loss: 0.1251
Epoch 33/60
23/23 [==============================] - 18s 782ms/step - loss: 0.1250
Epoch 34/60
23/23 [==============================] - 18s 779ms/step - loss: 0.1252
Epoch 35/60
23/23 [==============================] - 18s 781ms/step - loss: 0.1250
Epoch 36/60
23/23 [==============================] - 18s 779ms/step - loss: 0.1250
Epoch 37/60
23/23 [==============================] - 18s 783ms/step - loss: 0.1249
Epoch 38/60
23/23 [==============================] - 18s 781ms/step - loss: 0.1249
Epoch 39/60
23/2

23/23 [==============================] - 11s 458ms/step - loss: 0.0160
Epoch 6/60
23/23 [==============================] - 11s 457ms/step - loss: 0.0149
Epoch 7/60
23/23 [==============================] - 10s 456ms/step - loss: 0.0142
Epoch 8/60
23/23 [==============================] - 11s 457ms/step - loss: 0.0135
Epoch 9/60
23/23 [==============================] - 10s 456ms/step - loss: 0.0136
Epoch 10/60
23/23 [==============================] - 11s 459ms/step - loss: 0.0128
Epoch 11/60
23/23 [==============================] - 11s 457ms/step - loss: 0.0120
Epoch 12/60
23/23 [==============================] - 10s 457ms/step - loss: 0.0118
Epoch 13/60
23/23 [==============================] - 10s 456ms/step - loss: 0.0117
Epoch 14/60
23/23 [==============================] - 10s 456ms/step - loss: 0.0114
Epoch 15/60
23/23 [==============================] - 11s 457ms/step - loss: 0.0110
Epoch 16/60
23/23 [==============================] - 11s 457ms/step - loss: 0.0106
Epoch 17/60
23/23 [=

23/23 [==============================] - 11s 477ms/step - loss: 0.0048
Epoch 44/60
23/23 [==============================] - 11s 476ms/step - loss: 0.0056
Epoch 45/60
23/23 [==============================] - 11s 478ms/step - loss: 0.0053
Epoch 46/60
23/23 [==============================] - 11s 476ms/step - loss: 0.0049
Epoch 47/60
23/23 [==============================] - 11s 477ms/step - loss: 0.0047
Epoch 48/60
23/23 [==============================] - 11s 476ms/step - loss: 0.0046
Epoch 49/60
23/23 [==============================] - 11s 477ms/step - loss: 0.0046
Epoch 50/60
23/23 [==============================] - 11s 478ms/step - loss: 0.0044
Epoch 51/60
23/23 [==============================] - 11s 476ms/step - loss: 0.0044
Epoch 52/60
23/23 [==============================] - 11s 479ms/step - loss: 0.0044
Epoch 53/60
23/23 [==============================] - 11s 478ms/step - loss: 0.0045
Epoch 54/60
23/23 [==============================] - 11s 476ms/step - loss: 0.0046
Epoch 55/60
23/2

## Conclusions

* Training with different barcode orientation improves segmentation accuracy.

## References

<a id='ref_1'></a>
1. Alessandro Zamberletti, Ignazio Gallo, Moreno Carullo and Elisabetta Binaghi "Neural Image Restoration For Decoding 1-D Barcodes Using Common Camera Phones" Computer Vision, Imaging and Computer Graphics. Theory and Applications, Springer Berlin Heidelberg, 2011

<a id='ref_2'></a>
2. S. Wachenfeld, S. Terlunen, X.Jiang  "Robust recognition of 1-d barcodes using camera phones.""
In Proceedings of the 2008 19th International Conference on Pattern Recognition, Tampa, FL, USA,
8–11 December 2008; pp. 1–4.

<a id='ref_3'></a>
3. Alessandro Zamberletti, Ignazio Gallo and Simone Albertini "Robust Angle Invariant 1D Barcode Detection" Proceedings of the 2nd Asian Conference on Pattern Recognition (ACPR), Okinawa, Japan, 2013